#### This code mergs the two files aircrat_data_decimal and airways_data_decimal to create a single file.

In [19]:
import pandas as pd
import ast
from typing import Dict, List, Optional

def process_waypoints(row: pd.Series, required_waypoints: List[str]) -> List[Dict]:
    """
    Extract waypoint information for specific required waypoints.
    
    Parameters:
    -----------
    row : pd.Series
        Row from airways DataFrame containing waypoint information
    required_waypoints : List[str]
        List of waypoint names that need to be extracted
        
    Returns:
    --------
    List[Dict]
        List of dictionaries containing waypoint information
    """
    waypoints = []
    
    # Create a dictionary of all waypoints from the airway
    waypoint_dict = {}
    for i in range(1, 12):  # Airways data has up to 11 waypoints
        waypoint_name = row[f'Waypoint {i}']
        if pd.isna(waypoint_name):
            break
        
        waypoint_dict[waypoint_name] = {
            'name': waypoint_name,
            'latitude': row[f'Waypoint {i} Latitude (Decimal)'],
            'longitude': row[f'Waypoint {i} Longitude (Decimal)']
        }
    
    # Only include waypoints that are in the required list
    for waypoint in required_waypoints:
        if waypoint.strip() in waypoint_dict:
            waypoints.append(waypoint_dict[waypoint.strip()])
    
    return waypoints

def create_merged_dataset(aircraft_file: str, airways_file: str) -> pd.DataFrame:
    """
    Create a merged dataset from aircraft and airways data.
    
    Parameters:
    -----------
    aircraft_file : str
        Path to the aircraft data CSV file
    airways_file : str
        Path to the airways data CSV file
        
    Returns:
    --------
    pd.DataFrame
        Merged dataset containing aircraft and waypoint information
    """
    # Read the CSV files
    aircraft_df = pd.read_csv(aircraft_file)
    airways_df = pd.read_csv(airways_file)
    
    # Initialize list for merged data
    merged_data = []
    
    # Process each aircraft
    for _, aircraft in aircraft_df.iterrows():
        # Get the list of waypoints for this aircraft
        aircraft_waypoints = [wp.strip() for wp in aircraft['Air Routes'].split(',')]
        
        # Find the airway that contains all these waypoints
        matching_airway = None
        for _, airway in airways_df.iterrows():
            # Handle the string representation of the route list
            if isinstance(airway['Air Route'], str):
                if airway['Air Route'].startswith('['):
                    airway_route = [wp.strip().strip("'") for wp in airway['Air Route'].strip('[]').split(',')]
                else:
                    airway_route = [wp.strip() for wp in airway['Air Route'].split(',')]
                
                # Check if all aircraft waypoints are in this airway
                if all(wp in airway_route for wp in aircraft_waypoints):
                    matching_airway = airway
                    break
        
        if matching_airway is not None:
            waypoints = process_waypoints(matching_airway, aircraft_waypoints)
            
            merged_record = {
                'callsign': aircraft['Callsign'],
                'aircraft_type': aircraft['Aircraft Type'],
                'start_time': aircraft['Start Time'],
                'initial_latitude': aircraft['Initial Latitude (Decimal)'],
                'initial_longitude': aircraft['Initial Longitude (Decimal)'],
                'airway_name': matching_airway['Airway'],
                'waypoints': waypoints,
                'initial_flight_level': aircraft['Initial Altitude (FL)'],
                'final_flight_level': aircraft['Requested FL']
            }
            
            merged_data.append(merged_record)
    
    result_df = pd.DataFrame(merged_data)
    
    return result_df





In [20]:
merged_df = create_merged_dataset("aircraft_data_decimal_24jan.csv", "airways_data_decimal.csv")

In [21]:
merged_df 

,callsign,aircraft_type,start_time,initial_latitude,initial_longitude,airway_name,waypoints,initial_flight_level,final_flight_level
0,SQ10,B738,0,2.388333,103.871667,N884,"[{'name': 'VMR', 'latitude': 2.388333333333333...",400.0,350.0
1,SQ11,B738,150,2.388333,103.871667,N884,"[{'name': 'VMR', 'latitude': 2.388333333333333...",390.0,350.0
2,SQ12,B738,300,2.388333,103.871667,N884,"[{'name': 'VMR', 'latitude': 2.388333333333333...",380.0,350.0
3,SQ13,A320,100,3.408889,103.928889,M758,"[{'name': 'IDSEL', 'latitude': 3.4088888888888...",370.0,350.0
4,SQ14,A320,350,3.408889,103.928889,M758,"[{'name': 'IDSEL', 'latitude': 3.4088888888888...",360.0,350.0
5,SQ15,A320,600,3.408889,103.928889,M758,"[{'name': 'IDSEL', 'latitude': 3.4088888888888...",350.0,350.0
6,SQ16,B738,0,2.161111,107.845556,M761,"[{'name': 'SABIP', 'latitude': 2.1611111111111...",380.0,350.0
7,SQ17,B738,150,2.161111,107.845556,M761,"[{'name': 'SABIP', 'latitude': 2.1611111111111...",370.0,350.0
8,SQ18,B738,350,2.161111,107.845556,M761,"[{'name': 'SABIP', 'latitude': 2.1611111111111...",360.0,350.0
9,SQ19,B738,550,2.161111,107.845556,M761,"[{'name': 'SABIP', 'latitude': 2.1611111111111...",350.0,350.0


In [22]:
merged_df.loc[0]


callsign                                                             SQ10
aircraft_type                                                        B738
start_time                                                              0
initial_latitude                                                 2.388333
initial_longitude                                              103.871667
airway_name                                                          N884
waypoints               [{'name': 'VMR', 'latitude': 2.388333333333333...
initial_flight_level                                                400.0
final_flight_level                                                  350.0
Name: 0, dtype: object

In [17]:
merged_df.loc[0]['waypoints']

[{'name': 'VMR',
  'latitude': 2.388333333333333,
  'longitude': 103.87166666666666},
 {'name': 'LENDA', 'latitude': 2.69, 'longitude': 104.65},
 {'name': 'LIPRO', 'latitude': 2.89, 'longitude': 105.19},
 {'name': 'LEBIN', 'latitude': 3.24, 'longitude': 106.1},
 {'name': 'ONAPO', 'latitude': 3.35, 'longitude': 106.388},
 {'name': 'LUSMO',
  'latitude': 3.5613888888888887,
  'longitude': 106.92611111111113}]

In [23]:
# Save to CSV (optional)
merged_df.to_csv('merged_aircraft_airways_24jan.csv', index=False)

# Print the number of rows to verify
# print(f"Number of rows in merged dataset: {len(merged_df)}")

,callsign,aircraft_type,start_time,initial_latitude,initial_longitude,airway_name,waypoints,initial_flight_level,final_flight_level
0,SQ10,B738,300,2.388333,10.589167,N884,"[{'name': 'VMR', 'latitude': 2.388333333333333...",400.0,350.0
1,SQ11,B738,550,2.388333,10.589167,N884,"[{'name': 'VMR', 'latitude': 2.388333333333333...",400.0,350.0
2,SQ12,B734,200,2.388333,10.589167,M771,"[{'name': 'VMR', 'latitude': 2.388333333333333...",160.0,350.0
3,SQ13,B734,450,2.388333,10.589167,M771,"[{'name': 'VMR', 'latitude': 2.388333333333333...",160.0,350.0
4,SQ14,B734,700,2.388333,10.589167,M771,"[{'name': 'VMR', 'latitude': 2.388333333333333...",160.0,350.0
5,SQ15,B738,0,2.161111,11.251111,M761,"[{'name': 'SABIP', 'latitude': 2.1611111111111...",380.0,350.0
6,SQ16,B738,150,2.161111,11.251111,M761,"[{'name': 'SABIP', 'latitude': 2.1611111111111...",380.0,350.0
7,SQ17,B738,300,2.161111,11.251111,M761,"[{'name': 'SABIP', 'latitude': 2.1611111111111...",380.0,350.0
8,SQ18,B738,450,2.161111,11.251111,M761,"[{'name': 'SABIP', 'latitude': 2.1611111111111...",380.0,350.0
9,SQ19,B744,0,3.561389,11.098056,M758R,"[{'name': 'LUSMO', 'latitude': 3.5613888888888...",340.0,350.0


In [ ]:
merged_df.loc[0]